In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [ ]:
class NB:
    def prediksi(self, X):
        prediksi = [self._prediksi(i) for i in X]
        return np.array(prediksi)

    def _prediksi(self, x):
        posteriors = []
        for idx, nama_kelas in enumerate(self.kelas):
            prior = np.log(self.prior[idx])
            posterior = sum(np.log(self.fungsi_derivatif(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        return self.kelas[np.argmax(posteriors)]

    def fungsi_derivatif(self, kelas_indeks, x):
        mean = self.mean[kelas_indeks]
        var = self.var[kelas_indeks]
        pembilang = np.exp(-(x - mean) ** 2 / (2 * var))
        penyebut = np.sqrt(2 * np.pi * var)
        return pembilang / penyebut

    def fit(self, X, Y):
        n_baris, n_kolom = X.shape
        self.kelas = np.unique(Y)  # Add this line to initialize the 'kelas' attribute
        jumlah_kelas = len(self.kelas)

        self.mean = np.zeros((jumlah_kelas, n_kolom), dtype=np.float64)
        self.var = np.zeros((jumlah_kelas, n_kolom), dtype=np.float64)
        self.prior = np.zeros(jumlah_kelas, dtype=np.float64)

        for idx, nama_kelas in enumerate(self.kelas):
            X_kelas = X[Y == nama_kelas]
            self.mean[idx, :] = X_kelas.mean(axis=0)
            self.var[idx, :] = X_kelas.var(axis=0)
            self.prior[idx] = X_kelas.shape[0] / n_baris

In [ ]:
class FeatureSelectionGA:
    def __init__(self, model, feature_names, population_size=100, generations=50, crossover_prob=0.8, mutation_prob=0.2):
        self.model = model
        self.feature_names = feature_names
        self.population_size = population_size
        self.generations = generations
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob

    def _fitness(self, X_train, X_test, y_train, y_test, selected_features):
        model = self.model
        model.fit(X_train[:, selected_features], y_train)
        y_pred = model.prediksi(X_test[:, selected_features])
        return accuracy_score(y_test, y_pred)

    def _initialize_population(self, num_features):
        return np.random.choice([0, 1], size=(self.population_size, num_features))

    def _crossover(self, parents):
        children = []
        for i in range(0, len(parents), 2):
            parent1, parent2 = parents[i], parents[i + 1]
            if np.random.rand() < self.crossover_prob:
                crossover_point = np.random.randint(1, len(parent1) - 1)
                child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
                child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            else:
                child1, child2 = parent1.copy(), parent2.copy()
            children.extend([child1, child2])
        return np.array(children)

    def _mutation(self, children):
        mutation_mask = (np.random.rand(*children.shape) < self.mutation_prob).astype(int)
        return (children + mutation_mask) % 2

    def _select_features(self, X_train, X_test, y_train, y_test, population):
        fitness_values = []
        for features in population:
            fitness_values.append(self._fitness(X_train, X_test, y_train, y_test, np.where(features == 1)[0]))
        return np.array(fitness_values)

    def optimize(self, X_train, X_test, y_train, y_test):
        num_features = X_train.shape[1]
        population = self._initialize_population(num_features)

        for generation in tqdm(range(self.generations)):
            fitness_values = self._select_features(X_train, X_test, y_train, y_test, population)
            parents = population[np.argsort(fitness_values)[::-1][:self.population_size // 2]]

            children = self._crossover(parents)
            children = self._mutation(children)

            population = np.vstack([parents, children])

        best_features = np.where(population[0] == 1)[0]
        selected_feature_names = [self.feature_names[i] for i in best_features]
        return best_features, selected_feature_names


In [ ]:
# Load your dataset
df = pd.read_csv('breast_cancer.csv')
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

X = df.iloc[:, 2:].values
y = df.iloc[:, 1].values
feature_names = df.columns[2:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
display(df.head(10))

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN
5,843786,1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,NaN
6,844359,1,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,NaN
7,84458202,1,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,NaN
8,844981,1,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,NaN
9,84501001,1,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750,NaN


In [ ]:
#Count the number of rows and columns in the data set
df.shape

(569, 33)

In [ ]:
#Count the empty (NaN, NAN, na) values in each column
df.isna().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [ ]:
#Drop the column with all missing values (na, NAN, NaN)
#NOTE: This drops the column Unnamed
df = df.dropna(axis=1)

In [ ]:
#Get the new count of the number of rows and cols
df.shape

(569, 32)

In [ ]:
#Get a count of the number of 'M' & 'B' cells
df['diagnosis'].value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

In [ ]:
# Define Naive Bayes model
nb_model = NB()

In [ ]:
# Call the fit method to initialize the 'kelas' attribute
nb_model.fit(X_train, y_train)

In [ ]:
# Use genetic algorithm for feature selection
feature_selector = FeatureSelectionGA(model=nb_model, feature_names=feature_names)
best_features, selected_feature_names = feature_selector.optimize(X_train, X_test, y_train, y_test)

print("\n")
print("Best Features:", best_features)
print("Selected Feature Names:", selected_feature_names)
print("Number of Best Features:", len(best_features))
print("\n")

100%|██████████| 50/50 [00:44<00:00,  1.12it/s]



Best Features: [ 1  3  6  8  9 11 12 13 16 17 20 23 27]
Selected Feature Names: ['texture_mean', 'area_mean', 'concavity_mean', 'symmetry_mean', 'fractal_dimension_mean', 'texture_se', 'perimeter_se', 'area_se', 'concavity_se', 'concave points_se', 'radius_worst', 'area_worst', 'concave points_worst']
Number of Best Features: 13




In [ ]:
# Train Naive Bayes with the best features
nb_model.fit(X_train[:, best_features], y_train)
y_pred = nb_model.prediksi(X_test[:, best_features])
accuracy = accuracy_score(y_test, y_pred)
print(f"With feature selection, accuracy: {accuracy * 100:.2f}%")


With feature selection, accuracy: 98.25%


In [ ]:
# Train Naive Bayes without feature selection
nb_model_no_fs = NB()
nb_model_no_fs.fit(X_train, y_train)
y_pred_no_fs = nb_model_no_fs.prediksi(X_test)
accuracy_no_fs = accuracy_score(y_test, y_pred_no_fs)
print(f"Without feature selection, accuracy: {accuracy_no_fs * 100:.2f}%")

Without feature selection, accuracy: 62.28%


In [ ]:
# Create a new instance of the NB class and call the fit method
nb_model_with_fs = NB()
nb_model_with_fs.fit(X_train[:, best_features], y_train)

In [ ]:
def generate_random_data(num_features):
    return np.random.rand(1, num_features)

In [ ]:
num_features = X_train.shape[1]
X_random = generate_random_data(num_features)

print("Random Data:")
print(X_random)

Random Data:
[[0.83785722 0.26525053 0.40405929 0.251872   0.66128877 0.62764628
  0.35353845 0.71294679 0.26479442 0.91069983 0.7938598  0.55031061
  0.39616245 0.31840257 0.22260277 0.05769301 0.61816315 0.74478706
  0.77071867 0.65914231 0.16992527 0.91604697 0.09114761 0.06070817
  0.49158076 0.33777856 0.21286028 0.99214233 0.30400532 0.99417991
  0.00295672]]


In [ ]:
# Predict the diagnosis using the trained Naive Bayes model with feature selection
diagnosis_prediction = nb_model_with_fs.prediksi(X_random[:, best_features])

<ipython-input-2-701c9faa7b04>:10: RuntimeWarning: divide by zero encountered in log
  posterior = sum(np.log(self.fungsi_derivatif(idx, x)))


In [ ]:
# Map the prediction back to the original labels (if needed)
predicted_diagnosis_label = 'M' if diagnosis_prediction[0] == 1 else 'B'

print("\n")
print("Random Data for Prediction:")
print(X_random)
print("Predicted Diagnosis:", predicted_diagnosis_label)
print("\n")



Random Data for Prediction:
[[0.83785722 0.26525053 0.40405929 0.251872   0.66128877 0.62764628
  0.35353845 0.71294679 0.26479442 0.91069983 0.7938598  0.55031061
  0.39616245 0.31840257 0.22260277 0.05769301 0.61816315 0.74478706
  0.77071867 0.65914231 0.16992527 0.91604697 0.09114761 0.06070817
  0.49158076 0.33777856 0.21286028 0.99214233 0.30400532 0.99417991
  0.00295672]]
Predicted Diagnosis: B


